In [1]:
import warnings
import os
from glob import glob
import numpy as np
np.random.seed(1337)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.mlab as mlab
from scipy.stats import norm
from keras.utils.vis_utils import model_to_dot
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Layer
from keras.layers import Conv1D, Conv2D, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras import metrics
import keras
from skimage.transform import resize
from PIL import Image

import sys, os
from vae_tools import plot_model, layers, nb_tools, viz, loader, build_model, sanity, sampling, custom_variational_layer, mmvae
import vae_tools
import vae_tools.callbacks
sanity.check()


Using TensorFlow backend.


keras version: 2.2.4
tensorflow version: 1.12.0
matplotlib uses:  module://ipykernel.pylab.backend_inline
Available GPUs ['/job:localhost/replica:0/task:0/device:GPU:0']


In [2]:
# Load the data
steps_around_center, angles_around_center = loader.get_steps_around_hokuyo_center(degree_around_center = 80.)
image_rows_cols_chns = (64, 64, 2)
image_original_rows_cols_chns= (800,800,3)
new_shape = image_rows_cols_chns
old_shape = image_original_rows_cols_chns
measurements_per_file = 111 # 2018-06-02 dataset
measurements_per_file = 300 # 2018-06-05 dataset
# folder = glob('2018-06-02/cyl_r*/') + glob('2018-06-02/box_r*/');
# folder = glob('2018-06-05/cyl_r*/') + glob('2018-06-05/box_r*/') + glob('2018-06-05/cyl_g*/');
folder = ('../../mVAE/2018-06-05/cyl_r*/', '../../mVAE/2018-06-05/box_r*/', '../../mVAE/2018-06-05/cyl_g*/');
X_l, X_c, X_set_label_idx, X_set_label_str = loader.camera_lidar("./Xl-80deg_Xc-64-64-2", folder, 
                    "_amiro1_sync_front_camera_image_raw-X-pixeldata.npz", 
                    "_amiro1_sync_laser_scan-X-ranges_intensities_angles.npz", 
                    measurements_per_file, old_shape, new_shape, 683, steps_around_center, 5., overwrite = False)

In [3]:
def get_lidar_mask(X_set, original_dim, empty_perc = 1.0):
    # Prepare mask to remove all data where the objects are to small (to avoid learning empty measurements) (100% i.e. take everything)
    mask = np.ones((len(X_set),), dtype=np.bool)
    for idx in np.arange(len(X_set)):
        if np.sum(X_set[idx,:] == 0.0) / original_dim > empty_perc:
            mask[idx] = False
    return mask

def get_camera_mask(X_set, original_dim, empty_perc = 0.70):
    # Prepare the mask for removing all data which has a certain amount of empty space (to avoid learning empty images)
    empty_perc = 0.70
    mask = np.ones((len(X_set),), dtype=np.bool)
    for idx in np.arange(len(X_set)):
        if np.sum(X_set[idx,:] == 0.0) / original_dim > empty_perc:
            mask[idx] = False
    return mask

def get_lidar_data(X_l, X_set_label_idx, original_dim, use_conv, mask = None, X_set_random_idx = None, X_train_size = None):
    # Data normalization
    X_set = np.copy(X_l)
    X_set = X_set / np.amax(X_set)
    X_set_label = np.copy(X_set_label_idx)
    if mask is None:
        mask = get_lidar_mask(X_set, original_dim, empty_perc = 1.0)

    # normalization (min-0/max-1 and signal spreading)
    if True:
        for idx in np.arange(len(X_set)):
            min_val = np.amin(X_set[idx,X_set[idx,:] != 0.0])
            X_set[idx,X_set[idx,:] != 0.0] = X_set[idx,X_set[idx,:] != 0.0] - min_val
            max_val = np.amax(X_set[idx,:])
            X_set[idx,:] = X_set[idx,:] / max_val
            #
            X_set_val = X_set[idx,X_set[idx,:] != 0.0]
            num_values = len(X_set_val)
            idx_interp = np.linspace(0, num_values, num=227, endpoint=True) 
            idx_orig = np.linspace(0, num_values, num=num_values, endpoint=True)
            X_set[idx,:] = np.interp(idx_interp, idx_orig, X_set_val)

    # Shape for the input
    if use_conv:
        # remove the last value to make the number of measurements even
        X_set = X_set[:,:-1]
        # Add extra dimension
        X_set = np.expand_dims(X_set, axis=-1)

    # Remove unwanted the data
    X_set = X_set[mask,:]
    X_set_label = X_set_label[mask]

    # Get training and test set
    if X_set_random_idx is None:
        X_set_random_idx = np.arange(X_set.shape[0])
        np.random.shuffle(X_set_random_idx)
    if X_train_size is None:
        X_train_size = np.int(len(X_set) * 0.99)
    X_train = X_set[X_set_random_idx[:X_train_size],:]
    X_test = X_set[X_set_random_idx[X_train_size:],:]
    X_train_label = X_set_label[X_set_random_idx[:X_train_size]]
    X_test_label = X_set_label[X_set_random_idx[X_train_size:]]
    # Check normalization
    print("Minimum value: " + str(np.amin(X_set)))
    print("Maximum value: " + str(np.amax(X_set)))
    return X_train, X_test, X_train_label, X_test_label

def get_camera_data(X_c, X_set_label_idx, original_dim, use_conv, mask = None, X_set_random_idx = None, X_train_size = None):
    # Format the data for training
    X_set_label = np.copy(X_set_label_idx)
    X_set = np.reshape(np.copy(X_c), (len(X_c), image_rows_cols_chns[0], image_rows_cols_chns[1], image_rows_cols_chns[2]))
    X_set = 1. - X_set # Invert the brightness
    ## Normalize: Threshold uninformative (< 0.5 = 0), max=1
    X_set[X_set < 0.5] = 0
    X_set = X_set / np.amax(X_set)
    # Copy the upper half of the image to the lower one, to fill the floor with the object as well (to get more information)
    for idx in np.arange(len(X_set)):
        X_set[idx,int(new_shape[0]/2):,:,:] = np.flipud(X_set[idx,:int(new_shape[0]/2),:,:])

    if mask is None:
        mask = get_camera_mask(X_set, original_dim, empty_perc = 0.70)

    # Flatten the data if we do not use conv layers
    if not use_conv:
        X_set = X_set.reshape(-1,np.prod(image_rows_cols_chns))
    # Check normalization
    print("Minimum value: " + str(np.amin(X_set)))
    print("Maximum value: " + str(np.amax(X_set)))

    # Remove
    X_set = X_set[mask,:]
    X_set_label = X_set_label[mask]


    # define train and test set
    if X_set_random_idx is None:
        X_set_random_idx = np.arange(X_set.shape[0])
        np.random.shuffle(X_set_random_idx)
    if X_train_size is None:
        X_train_size = np.int(len(X_set) * 0.99)
    X_train = X_set[X_set_random_idx[:X_train_size],:]
    X_test = X_set[X_set_random_idx[X_train_size:],:]
    X_train_label = X_set_label[X_set_random_idx[:X_train_size]]
    X_test_label = X_set_label[X_set_random_idx[X_train_size:]]

    return X_train, X_test, X_train_label, X_test_label

def get_data(X_c, X_l, X_set_label_idx, original_dim_lidar, original_dim_camera, use_conv_lidar, use_conv_camera):
    mask = np.logical_and(get_camera_mask(X_c, original_dim_camera, empty_perc = 0.70), get_lidar_mask(X_l, original_dim_lidar, empty_perc = 1.0))
    num_samples = np.sum(mask)
    X_set_random_idx = np.arange(num_samples) #  no randomnes
    np.random.shuffle(X_set_random_idx)
    X_train_size = np.int(num_samples * 0.99)
    X_c_train, X_c_test, X_c_train_label, X_c_test_label = get_camera_data(X_c, X_set_label_idx, original_dim_camera, use_conv_camera, mask = mask, X_set_random_idx = X_set_random_idx, X_train_size = X_train_size)
    X_l_train, X_l_test, _, _ = get_lidar_data(X_l, X_set_label_idx, original_dim_lidar, use_conv_lidar, mask = mask, X_set_random_idx = X_set_random_idx, X_train_size = X_train_size)
    return X_l_train, X_l_test, X_c_train, X_c_test, X_c_train_label, X_c_test_label


In [4]:
def vae_lidar(train = True, use_conv = False, X_set = None, epochs = 500):
    
    # Setup the VAE
    batch_size = 256
    if use_conv:
        original_dim = 226
        # Conv settings
        # number of convolutional filters to use
        filters = 16
        # convolution kernel size
        num_conv = 5
    else:
        original_dim = 227
    intermediate_dim = 128
    original_dim_2 = int(original_dim / 2)
    intermediate_dim_2 = int(intermediate_dim / 2)
    latent_dim = 2
    epsilon_std = 1.
    beta = 0.125
    # beta = 0.01 # works as well

    if use_conv:
        class Conv1DTranspose():
            def __init__(self, filters, kernel_size, strides=2, activation='relu', padding='same'):
                self.input  = x = Lambda(lambda x: K.expand_dims(x, axis=2))
                self.conv   = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding)
                self.output = Lambda(lambda x: K.squeeze(x, axis=2))
            def __call__(self, input_tensor):
                return self.output(self.conv(self.input(input_tensor)))

        encoder = [[
            Input(shape=(original_dim,1)),
            Conv1D(1, kernel_size=2, padding='same', activation='relu'),
            Conv1D(filters, kernel_size=2, padding='same', activation='relu', strides=2),
            Conv1D(filters, kernel_size=num_conv, padding='same', activation='relu', strides=1),
            Conv1D(filters, kernel_size=num_conv, padding='same', activation='relu', strides=1),
            Flatten(),
            Dense(intermediate_dim, activation='relu')
        ]]

        output_shape = (batch_size, original_dim_2, filters)

        decoder = [[
            Dense(intermediate_dim, activation='relu'),
            Dense(filters * original_dim_2, activation='relu'),
            Reshape(output_shape[1:]),
            Conv1DTranspose(filters, kernel_size=num_conv, padding='same', strides=1, activation='relu'),
            Conv1DTranspose(filters, kernel_size=num_conv, padding='same', strides=1, activation='relu'),
            Conv1DTranspose(filters, kernel_size=3, strides=2, padding='valid', activation='relu'),
            Conv1D(1, kernel_size=2, padding='valid', activation='sigmoid')
        ]]
    else:
        encoder = [[
            Input(shape=(original_dim,)),
            Dense(intermediate_dim, activation='relu'),
            Dense(intermediate_dim_2, activation='relu')
        ]]

        decoder = [[
            Dense(intermediate_dim_2, activation='relu'),
            Dense(intermediate_dim, activation='relu'),
            Dense(original_dim, activation='sigmoid')
        ]]

    model_obj = vae_tools.mmvae.MmVae(latent_dim, encoder, decoder, [original_dim], beta, reconstruction_loss_metrics = [vae_tools.mmvae.ReconstructionLoss.MSE], name='Vae')
    vae = model_obj.get_model()
    vae.compile(optimizer='rmsprop', loss=None)
    # vae_tools.viz.plot_model(vae, file = 'myVAE', print_svg = False, verbose = True)

    if X_set is None:
        X_train, X_test, X_train_label, X_test_label = get_lidar_data(X_l, X_set_label_idx, original_dim, use_conv)
    else:
        X_train, X_test, X_train_label, X_test_label = X_set["X_train"], X_set["X_test"], X_set["X_train_label"], X_set["X_test_label"]
        
    
    # Train
    if train:
        vae.fit(X_train,
                shuffle=True,
                epochs=epochs,
                batch_size=batch_size,
                verbose = 0,
                validation_data=(X_test, None))

    return model_obj, use_conv

In [5]:
def vae_camera(train = True, use_conv = False, X_set = None, epochs = 1000):

    # Setup the VAE
    batch_size = 256
    # input image dimensions
    image_rows_cols_chns = (64, 64, 2)
    if use_conv:
        # number of convolutional filters to use
        filters = 32 # former 64
        # convolution kernel size
        num_conv = 3
        # Other values
        rows = image_rows_cols_chns[0]
        rows_2 = int(rows/2)
        cols = image_rows_cols_chns[1]
        cols_2 = int(cols/2)
        img_chns = image_rows_cols_chns[2]
        if keras.backend.image_data_format() == 'channels_first':
            original_img_size = (image_rows_cols_chns[2], image_rows_cols_chns[0], image_rows_cols_chns[1])
            output_shape_reshape = (batch_size, filters, rows_2, cols_2)
            output_shape_upsamp = (batch_size, filters, rows+1, cols+1)
        else:
            original_img_size = image_rows_cols_chns
            output_shape_reshape = (batch_size, rows_2, cols_2, filters)
            output_shape_upsamp = (batch_size, rows+1, cols+1, filters)
    original_dim = np.prod(image_rows_cols_chns)
    latent_dim = 2
    intermediate_dim = 256 # former 128
    intermediate_dim_2 = int(intermediate_dim / 2)
    save_model = False
    beta = 50.

    if use_conv:
        encoder = [[
            Input(shape=original_img_size),
            Conv2D(img_chns, kernel_size=(2, 2), padding='same', activation='relu'),
            Conv2D(filters, kernel_size=(2, 2), padding='same', activation='relu', strides=(2, 2)),
            Conv2D(filters, kernel_size=num_conv, padding='same', activation='relu', strides=1),
            Conv2D(filters, kernel_size=num_conv, padding='same', activation='relu', strides=1),
            Flatten(),
            Dense(intermediate_dim, activation='relu')
        ]]

        decoder = [[
            Dense(intermediate_dim, activation='relu'),
            Dense(filters * rows_2 * cols_2, activation='relu'),
            Reshape(output_shape_reshape[1:]),
            Conv2DTranspose(filters, kernel_size=num_conv, padding='same', strides=1, activation='relu'),
            Conv2DTranspose(filters, kernel_size=num_conv, padding='same', strides=1, activation='relu'),
            Conv2DTranspose(filters, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu'),
            Conv2D(img_chns, kernel_size=2, padding='valid', activation='sigmoid')
        ]]

    else:
        encoder = [[
            Input(shape=(original_dim,)),
            Dense(intermediate_dim, activation='relu'),
            Dense(intermediate_dim_2, activation='relu')
        ]]

        decoder = [[
            Dense(intermediate_dim_2, activation='relu'),
            Dense(intermediate_dim, activation='relu'),
            Dense(original_dim, activation='sigmoid')
        ]]
        
    model_obj = vae_tools.mmvae.MmVae(latent_dim, encoder, decoder, [original_dim], beta, reconstruction_loss_metrics = [vae_tools.mmvae.ReconstructionLoss.MSE], name='Vae')
    vae = model_obj.get_model()
    vae.compile(optimizer='rmsprop', loss=None)
    # vae_tools.viz.plot_model(vae, file = 'myVAE', print_svg = False, verbose = True)
    
    if X_set is None:
        X_train, X_test, X_train_label, X_test_label = get_camera_data(X_c, X_set_label_idx, original_dim, use_conv)
    else:
        X_train, X_test, X_train_label, X_test_label = X_set["X_train"], X_set["X_test"], X_set["X_train_label"], X_set["X_test_label"]
        
    
    # Train
    if train:
        vae.fit(X_train,
                shuffle=True,
                epochs = epochs,
                batch_size = batch_size,
                verbose = 0,
                validation_data=(X_test, None))

    
    return model_obj, use_conv

In [6]:
model_lidar, use_conv_lidar = vae_lidar(train = False, use_conv = True)
model_camera,use_conv_camera = vae_camera(train = False, use_conv = True)
#model_lidar, use_conv_lidar = vae_lidar(train = True, use_conv = True, epochs = 10)
#model_camera,use_conv_camera = vae_camera(train = True, use_conv = True, epochs = 10)
X_l_train, X_l_test, X_c_train, X_c_test, X_train_label, X_test_label = get_data(X_c, X_l, X_set_label_idx, 
                                                                                     model_lidar.encoder_inputs_dim[0], model_camera.encoder_inputs_dim[0],
                                                                                     use_conv_lidar, use_conv_camera)
model_lidar, use_conv_lidar = vae_lidar(train = True, use_conv = True, epochs = 500, X_set = {"X_train": X_l_train, "X_test": X_l_test, "X_train_label": X_train_label, "X_test_label": X_test_label})
model_camera,use_conv_camera = vae_camera(train = True, use_conv = True, epochs = 1000, X_set = {"X_train": X_c_train, "X_test": X_c_test, "X_train_label": X_train_label, "X_test_label": X_test_label})



Minimum value: 6.7945444e-07
Maximum value: 1.0
Minimum value: 0.0
Maximum value: 1.0
Minimum value: 0.0
Maximum value: 1.0
Minimum value: 6.7945444e-07
Maximum value: 1.0


KeyboardInterrupt: 

In [ ]:
print(X_c_train[1].shape)
image =X_c_train[1]
image = (255. - np.concatenate((image, np.ones((image.shape[1],image.shape[0],1))), axis=2) * 255.) / 255.
plt.imshow(image)

In [ ]:
data = [ model_lidar.get_encoder_mean([model_lidar.encoder[0][0]]).predict(X_l_train),
        model_camera.get_encoder_mean([model_camera.encoder[0][0]]).predict(X_c_train)]
data_test = [ model_lidar.get_encoder_mean([model_lidar.encoder[0][0]]).predict(X_l_test),
        model_camera.get_encoder_mean([model_camera.encoder[0][0]]).predict(X_c_test)]


idx = 1
print(data[idx].shape)
plt.scatter(data[idx][:,0], data[idx][:,1], c = X_train_label)
plt.show()
print(data[idx][:,0])
print(model_camera.get_encoder_logvar([model_camera.encoder[0][0]]).predict(X_c_train[[0]]))

In [ ]:
# Multi modal training of proprio and voxel
data = [ model_lidar.get_encoder_mean([model_lidar.encoder[0][0]]).predict(X_l_train),
        model_camera.get_encoder_mean([model_camera.encoder[0][0]]).predict(X_c_train)]
data_test = [ model_lidar.get_encoder_mean([model_lidar.encoder[0][0]]).predict(X_l_test),
        model_camera.get_encoder_mean([model_camera.encoder[0][0]]).predict(X_c_test)]
model_name = "lidar_camera_vae"
store_model = True # Train and store the model if "True" or loads existing models if "False"
store_model_overwrite = False # Overwrite model if it exists
store_model_prefix = "models/lidar_camera" # prefix for loading and storing
if store_model:

    batch_size = 64
    epochs = 20
    z_dim = 2
    beta_norm = 0.01
    original_dim = 2
    intermediate_dim = 64
    intermediate_dim_2 = int(intermediate_dim / 2)
    epsilon_std = 1.
    beta = 0.125

    encoder_pv = [[
            Input(shape=(original_dim,)),
            Dense(intermediate_dim, activation='relu'),
            Dense(intermediate_dim_2, activation='relu')
        ],[
            Input(shape=(original_dim,)),
            Dense(intermediate_dim, activation='relu'),
            Dense(intermediate_dim_2, activation='relu')
        ]]
    decoder_pv = [[
            Dense(intermediate_dim_2, activation='relu'),
            Dense(intermediate_dim, activation='relu'),
            Dense(original_dim, activation='sigmoid')
        ],[
            Dense(intermediate_dim_2, activation='relu'),
            Dense(intermediate_dim, activation='relu'),
            Dense(original_dim, activation='sigmoid')
        ]]

    vae_obj = vae_tools.mmvae.MmVae(z_dim, encoder_pv, decoder_pv, [2, 2],
                    beta_norm, beta_is_normalized = True, reconstruction_loss_metrics = [vae_tools.mmvae.ReconstructionLoss.MSE], name=model_name)

    vae = vae_obj.get_model()
    vae.compile(optimizer='adam', loss=None)
    #vae_tools.viz.plot_model(vae, file = 'myVAE', print_svg = False, verbose = True)
    # Store the losses, encodings, and decodings to tensorboard
    losses_cb = vae_tools.callbacks.Losses(data = data_test)
    # Train
    vae.fit(data, shuffle=True, epochs=epochs, batch_size=batch_size, validation_data=None, verbose = 1, callbacks=[losses_cb])
    # Show the losses
    try:
        vae_tools.viz.plot_losses(losses_cb, plot_elbo = True)
    except:
        pass
   # Get the models
    encoder_input = [encoder_pv[0][0], encoder_pv[1][0]]
    encoder_mean = [vae_obj.get_encoder_mean([encoder_input[0]]),
                                 vae_obj.get_encoder_mean([encoder_input[1]]),
                                 vae_obj.get_encoder_mean(encoder_input)]
    encoder_logvar = [vae_obj.get_encoder_logvar([encoder_input[0]]),
                                   vae_obj.get_encoder_logvar([encoder_input[1]]),
                                   vae_obj.get_encoder_logvar(encoder_input)]
    
    # Store model
    #vae_obj.store_model(name = store_model_prefix + "_" + model_name + "_dec", model = vae_obj.get_decoder(), overwrite = store_model_overwrite)
    vae_obj.store_model_powerset(prefix = store_model_prefix + "_" + model_name + "_enc_mean_ProprioVoxel",
                                 model_inputs = encoder_input,
                                 get_model_callback = vae_obj.get_encoder_mean, overwrite = store_model_overwrite)
    vae_obj.store_model_powerset(prefix = store_model_prefix + "_" + model_name + "_enc_logvar_ProprioVoxel",
                                 model_inputs = encoder_input,
                                 get_model_callback = vae_obj.get_encoder_logvar, overwrite = store_model_overwrite)
else:
    # Load the models
    #decoder_mean = MmVae.load_model(name = store_model_prefix + "_" + model_name + "_dec")
    encoder_mean, _ =  vae_tools.mmvae.MmVae.load_model_powerset(prefix = store_model_prefix + "_" + model_name + "_enc_mean_ProprioVoxel",
                                             num_elements = 2)
    encoder_logvar, _ =  vae_tools.mmvae.MmVae.load_model_powerset(prefix = store_model_prefix + "_" + model_name + "_enc_logvar_ProprioVoxel",
                                               num_elements = 2)



In [ ]:
import vae_tools
from importlib import reload
reload(vae_tools.viz)
mean_encoded = encoder_mean[0].predict(data[0])
vae_tools.viz.plot_embedding(mean_encoded, X_train_label, figsize=(6, 6), colormap=None, show_ticks = True)
plt.axis("equal")
plt.title("lidar")

mean_encoded = encoder_mean[1].predict(data[1])
vae_tools.viz.plot_embedding(mean_encoded, X_train_label, figsize=(6, 6), colormap=None, show_ticks = True)
plt.axis("equal")
plt.title("camera")

# Plot the embedding proprio and voxel
mean_encoded = encoder_mean[2].predict(data)
#var_encoded = np.sum(np.exp(encoder_logvar[2].predict(data, batch_size=batch_size)),axis=-1) # trace of co-variance
#vae_tools.viz.plot_embedding(mean_encoded, var_encoded, colormap="rainbow", figsize=(6, 6))
#plt.colorbar()
vae_tools.viz.plot_embedding(mean_encoded, X_train_label, figsize=(6, 6), colormap=None, show_ticks = True)
plt.axis("equal")
plt.title("lidar/camera")
plt.show()

In [ ]:
idx = 14
print(X_l.shape)
plt.plot(X_l[idx])
#plt.plot(data[0][idx])
plt.show()
image = data[1][idx]
if len(image.shape) == 1:
    image = np.matlib.reshape(image, (64,64,2))
image = (255. - np.concatenate((image, np.ones((image.shape[1],image.shape[0],1))), axis=2) * 255.) / 255.
plt.imshow(image)

In [ ]:
model_idx = 0 # 0:lidar, 1: camera
X_train = data[model_idx]
encoder = encoder_mean[model_idx]
encoder_lv = encoder_logvar[model_idx]
if model_idx == 0:
    decoder = vae_obj.get_decoder(decoder_output_list = [model_lidar.decoder[0][-1]])
else:
    decoder = vae_obj.get_decoder(decoder_output_list = [model_camera.decoder[0][-1]])

# Vizualization
# Encode samples to get the min and max values in latent space
X_encoded = encoder.predict(X_train, batch_size=batch_size)

# Plot in- and output signals
if model_idx == 0:
    viz.lidar_in_out_2(X_train, 16, encoder, decoder)

# display a 2D manifold
nx = 50
ny = 50

# Chose linearly spaced coordinates according to the above figure
grid_x = (np.linspace(np.min(X_encoded[:, 0]), np.max(X_encoded[:, 0]), nx))
grid_y = (np.linspace(np.min(X_encoded[:, 1]), np.max(X_encoded[:, 1]), ny))
grid_x = norm.ppf(np.linspace(0.001, 0.999, nx))
grid_y = norm.ppf(np.linspace(0.001, 0.999, ny))
grid_x = np.linspace(np.amin(grid_x), np.amax(grid_x), nx)
grid_y = np.linspace(np.amin(grid_y), np.amax(grid_y), ny)

# Project input to latent space
viz.scatter_encoder_2(X_train, X_train_label, grid_x, grid_y, encoder, figsize=(15, 15), dpi=150)

# Plot the resampled inputs
if model_idx == 0:
    figures, z_reencoded_mean, z_reencoded_std = viz.get_lidar_dec_enc_samples_2(grid_x, grid_y, encoder, encoder_lv, decoder, Dz = 2)
    plt.show()
else:
    figure, z_reencoded_mean, z_reencoded_std = viz.get_image_dec_enc_samples_2(grid_x, grid_y, encoder, encoder_lv, decoder, Dz = 2, image_size = image_rows_cols_chns)
    plt.figure(figsize=(15, 15), dpi=96)
    plt.imshow(figure, cmap='Greys_r')
    plt.show()

# Plot the resampled std deviations
X, Y = np.meshgrid(np.arange(0,len(grid_x)), np.arange(0,len(grid_y)))
plt.pcolor(X, Y, z_reencoded_std, cmap='coolwarm', vmin=z_reencoded_std.min(), vmax=z_reencoded_std.max())
plt.colorbar()
plt.axis("image")
plt.show()